In [1]:
import os
import shutil
import sys
from collections import Counter
from copy import deepcopy
from pathlib import Path

abs_module_path = Path("./../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.data.processeddatainstance import ProcessedDataInstance
from modules.shared.config import load_config
from modules.shared.utils import create_new_dir, exclude_paths, get_repo_root

In [ ]:
processed_di = ProcessedDataInstance()
processed_di.parse_config("ml_analysis.toml")

In [ ]:
root = processed_di.palmskin_processed_dir
subdirs = exclude_paths(list(root.glob(f"*/SLIC/*")), [".tif"])

dir_name_cnt = Counter()
for subdir in subdirs:
    dir_name = subdir.parts[-1]
    dir_name_cnt.update([dir_name])

dir_name_cnt

In [ ]:
repo_root = get_repo_root()
collect_dst = repo_root.joinpath("script_ml/data/collected/SLIC_analyze")


for dir_name in dir_name_cnt.keys():
    
    src_dir = collect_dst.joinpath(processed_di.instance_name, dir_name)
    create_new_dir(src_dir)
    
    paths = list(root.glob(f"*/SLIC/{dir_name}/*.seg1.png"))
    for path in paths:
        
        dname = path.relative_to(root).parts[0]
        path_split = deepcopy(list(path.parts))
        path_split[-1] = path_split[-1].replace(".seg1.png", ".ana.toml")
        toml_file = Path(*path_split)
        cell_cnt = load_config(toml_file)["cell_count"]
        
        new_path = src_dir.joinpath(f"cnt{cell_cnt}_{dname}.png")
        print(new_path)
        shutil.copy(path, new_path)